In [ ]:
%pip install chromadb

In [ ]:
%pip install sentence_transformers

In [ ]:
from tinydb import TinyDB, Query

db = TinyDB('db.json')
table = db.table('articles')

articles = table.all()

print(f'loaded {len(articles)} articles')

In [ ]:
articles = [x for x in articles if x['abstract'] != 'No abstract available.']

print(f'retaining {len(articles)} articles')

In [ ]:
documents = []
ids = []

for article in articles:
    documents.append(article['abstract'])
    ids.append(article['link'])

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

client = chromadb.PersistentClient(path="vectors_db")

ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="allenai-specter")

# client.delete_collection(name="articles")
collection = client.create_collection(
    name="articles",
    embedding_function=ef
)

collection.add(
    documents=documents,
    ids=ids
)

In [ ]:
results = collection.query(
    query_texts=["infectious diseases transmitted by mosquitoes affecting children"],
    n_results=10
)

results